# Constellation and Chain Analysis: MultiConstellation Multihop MultiPath

<img src="MultipleConstellations.jpg" alt="Drawing" style="width: 500px;"/>

**Terminology**
* Node = Object in STK
* Edge = Access between two objects in STK
* Strand = The sequence of nodes and edges to complete access in a chain

**This notebook shows how to:**
* Find the shortest path between a starting and ending constellation, with many potential intermediate constellations.
* A chain will be built between each sequential pair in the constellationOrder list. Then networkx will be used build the network with the nodes coming from the constellations and the connections between the nodes coming from the chain accesses. Multiple sublists can be passed into constellationOrderList.
* Typical STK constraints such as range, link duration, Eb/No, etc are taken into account
* Data in the df variable can be pushed back into STK as a user supplied variable, a strand can be shown using object lines, and active objects over the analysis time period or at a time instance can be turned on.

**To reduce the runtime on subsequent runs, this scripts saves various computations to binary files and are reused for subsequent runs.** 
* The strands from the chains will be saved in the SavedNodes folder
* the nodes and associated time delays will be saved in the SavedNodes folder
* the node positions over time are saved in the SavedPositions folder
* the accesses between nodes over time are saved in the SavedEdges folder. 
* These folders will be created as subfolders of the directory used to run the script.
* During the first run the files will automatically be built and saved, subsequent runs will reload these files.
* To make changes simply delete the associated .pkl file for any changed strands, nodes, etc. and the script will recompute the data as needed. Or force all of the data to be overridden by setting the override options to be True.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('max_colwidth', 120)
from comtypes.client import CreateObject
from comtypes.client import GetActiveObject
from comtypes.gen import STKObjects
from comtypes.gen import STKUtil
from comtypes.gen import AgSTKVgtLib
import seaborn as sns
import matplotlib.pyplot as plt
from chainPathLib2 import *
import time
import networkx as nx
folders = ['SavedNodes','SavedPositions','SavedStrands','SavedEdges','SavedNetworkData']
for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)

## Constellation Connection Order, Computation Time, Metric, Saved Data Options

In [2]:
# The constellationOrderLists builds a directed chain for each adjacent pair in the list
# i.e.: constellationOrderLists = [[A,B],[X,Y,Z,X]] would build chainAB,chainXY,chainYZ,chainZX
constellationOrderLists = [['Targets','ObservingSatsFORs','ObservingSats','ObservingSatsTransmitters','ObservingSatsReceivers','ObservingSats','RelaySatsFORs','RelaySats','RelaySatsFORs','EndLocations'],['RelaySats','RelaySats']]   
startingConstellation = 'Targets' # complete path starting constellation
endingConstellation = 'EndLocations' # complete path ending constellation

start = 0 # EpSec
stop = 60*10 # EpSec
metric = 'timeDelay' # 'distance' or 'timeDelay' or 'bandwidth', custom metrics could be added
nodeDelays = {'ObservingSatsFORs':0.01,'ObservingSatsTransmitters':0.005,'ObservingSatsReceivers':0.005,'RelaySatsFORs':0.01,'RelaySats':0.002} # Add in time delays. Provide the constellation name in STK and the node delays
stkVersion = 12
overrideStrands = False # Override previously computed chains
overrideNodeDelaysByNode = False # Override previously built node delay dictionaries
overrideNodesTimesPos = False # Override previously built node positions
overrideNetwork = False # Override previously built node positions

In [3]:
# Connect to STK
stkApp = GetActiveObject('STK{}.Application'.format(stkVersion))
stkRoot = stkApp.Personality2
stkRoot.Isolate()
stkRoot.UnitPreferences.SetCurrentUnit('DateFormat','EpSec') # Units to EpSec for ease of use
stkRoot.ExecuteCommand('Units_SetConnect / Date "EpochSeconds"');
try:
    stkRoot.ExecuteCommand('VO * ObjectLine DeleteAll'); # Clean up old object lines
except:
    pass

# Build chains and create a dict of time delays for each node
t1 = time.time()
chainNames = createDirectedChains(stkRoot,constellationOrderLists,start=start,stop=stop,color=12895232)   
print(time.time()-t1)
t1 = time.time()
nodeDelaysByNode = getNodeDelaysByNode(stkRoot,nodeDelays,chainNames=chainNames,overrideData=overrideNodeDelaysByNode)
print(time.time()-t1)

1.2983002662658691
0.004064083099365234


## Compute  Strands and Distances

In [4]:
# Compute strands, (this is actually a list of all intervals for each edge)
t1 = time.time()
strands,dfStrands = getAllStrands(stkRoot,chainNames,start,stop,overrideData=overrideStrands)
print(time.time()-t1)
dfStrands

0.15595698356628418


,strand,start,stop,dur,num hops
827,"(Target/Target3, Satellite/ObservingSat63/Sensor/ObservingFOR)",0.000000,9.216490,9.216490,0.0
33844,"(Satellite/ObservingSat53, Satellite/RelaySat22/Sensor/RelaySatFOR)",0.000000,13.059304,13.059304,0.0
34466,"(Satellite/ObservingSat56, Satellite/RelaySat24/Sensor/RelaySatFOR)",0.000000,13.059304,13.059304,0.0
31332,"(Satellite/ObservingSat33, Satellite/RelaySat12/Sensor/RelaySatFOR)",0.000000,13.059306,13.059306,0.0
31954,"(Satellite/ObservingSat36, Satellite/RelaySat14/Sensor/RelaySatFOR)",0.000000,13.059306,13.059306,0.0
36296,"(Satellite/ObservingSat73, Satellite/RelaySat32/Sensor/RelaySatFOR)",0.000000,13.059327,13.059327,0.0
36918,"(Satellite/ObservingSat76, Satellite/RelaySat34/Sensor/RelaySatFOR)",0.000000,13.059327,13.059327,0.0
29083,"(Satellite/ObservingSat13, Satellite/RelaySat42/Sensor/RelaySatFOR)",0.000000,13.059329,13.059329,0.0
29705,"(Satellite/ObservingSat16, Satellite/RelaySat44/Sensor/RelaySatFOR)",0.000000,13.059329,13.059329,0.0
12936,"(Satellite/ObservingSat42/Transmitter/ObservingTransmitter, Satellite/ObservingSat24/Receiver/ObservingReceiver)",0.000000,28.625759,28.625759,0.0


In [5]:
# Compute node positions, distances and time delays

# Time resolution of distance/time computation
step = 10 # sec

t1 = time.time()
nodesTimesPos = computeNodesPosOverTime(stkRoot,strands,start,stop,step,overrideData=overrideNodesTimesPos) # Pull node position over time
t2 = time.time()
print(t2-t1)

t1 = time.time()
strandsAtTimes = getStrandsAtTimes(strands,start,stop,step) # Discretize strand intervals into times
t2 = time.time()
print(t2-t1)

t1 = time.time()
timeNodePos = computeTimeNodePos(strandsAtTimes,nodesTimesPos) # Nodes and positions at each time
t2 = time.time()
print(t2-t1)

t1 = time.time()
timesEdgesDistancesDelays = computeTimeEdgesDistancesDelays(strandsAtTimes,nodesTimesPos,nodeDelaysByNode,overrideData=True)  # Edges, distances and delays at each time
t2 = time.time()
print(t2-t1)

1.2964825630187988
0.47127246856689453
0.07999229431152344
0.6690800189971924


## Use NX for Network Metrics and Reliability Analysis


In [6]:
# Get pairs of each starting and ending node permutation in the constellations
startingNodes = getNodesFromConstellation(stkRoot,startingConstellation)
endingNodes = getNodesFromConstellation(stkRoot,endingConstellation)
nodePairs = [(start,end) for start,end in itertools.product(startingNodes, endingNodes)] # full permutation
pd.DataFrame(nodePairs)

,0,1
0,Target/Target1,Place/Los_Angeles_CA
1,Target/Target1,Place/Washington_DC
2,Target/Target2,Place/Los_Angeles_CA
3,Target/Target2,Place/Washington_DC
4,Target/Target3,Place/Los_Angeles_CA
5,Target/Target3,Place/Washington_DC
6,Target/Target4,Place/Los_Angeles_CA
7,Target/Target4,Place/Washington_DC
8,Target/Target5,Place/Los_Angeles_CA
9,Target/Target5,Place/Washington_DC


In [7]:
# Loop through each node pair and compute network metrics
# Edit computeNetworkMetrics in chainPathLibCustom to for additional metrics
for nodePair in nodePairs:
    df = computeNetworkMetrics(start,stop,step,timeNodePos,timesEdgesDistancesDelays,[nodePair[0]],[nodePair[1]],metric,computeNumNodesToLoseAccessBetweenAnyPair=True,overrideData=overrideNetwork,printTime=True,diNetwork=True)

0.004002571105957031
0.0030052661895751953
0.0020127296447753906
0.00197601318359375
0.0020074844360351562
0.0020194053649902344
0.001981019973754883
0.002019166946411133
0.0019795894622802734
0.0019991397857666016
0.0030002593994140625
0.0029990673065185547
0.006000518798828125
0.003000497817993164


##  Investigate Routing between Specific Nodes

In [8]:
# Pick a starting and ending node
startingNode = 'Target/Target7'
endingNode = 'Place/Washington_DC'

In [9]:
# Load df
filename = 'SavedNetworkData/df{}{}.pkl'.format(startingNode.split('/')[-1],endingNode.split('/')[-1])
with open(filename,'rb') as f:
    df = pickle.load(f)
df = addLightAndNodeDelays(df,timesEdgesDistancesDelays)    
dfIntervals = createDfIntervals(df,stop,step)
addStrandsAsObjectLines(stkRoot,dfIntervals,color='yellow')

In [10]:
# Add data back into STK for reporting and plotting
t1 = time.time()
df['distance'] = df['distance']*1000 # May need to fix meter/kilometer issue
addDataToSTK(stkRoot,chainNames[0],df) # Adds data in df back into STK to the first chain under User Supplied data
print(time.time()-t1)
df

0.158005952835083


,time,strand,timeDelay,num hops,num parent hops,Highest Num Nodes Removed To Lose Access,Lowest Num Nodes Removed To Lose Access,distance,lightDelay,nodeDelay
0,0,"[Target/Target7, Satellite/ObservingSat21/Sensor/ObservingFOR, Satellite/ObservingSat21, Satellite/ObservingSat21/Tr...",0.108642,6.0,3.0,3,3,2.357617e+07,0.078642,0.030
1,10,"[Target/Target7, Satellite/ObservingSat64/Sensor/ObservingFOR, Satellite/ObservingSat64, Satellite/ObservingSat64/Tr...",0.108554,6.0,3.0,3,3,2.354994e+07,0.078554,0.030
2,20,"[Target/Target7, Satellite/ObservingSat64/Sensor/ObservingFOR, Satellite/ObservingSat64, Satellite/ObservingSat64/Tr...",0.108469,6.0,3.0,3,3,2.352454e+07,0.078469,0.030
3,30,"[Target/Target7, Satellite/ObservingSat21/Sensor/ObservingFOR, Satellite/ObservingSat21, Satellite/ObservingSat21/Tr...",0.108504,6.0,3.0,3,3,2.353483e+07,0.078504,0.030
4,40,"[Target/Target7, Satellite/ObservingSat21/Sensor/ObservingFOR, Satellite/ObservingSat21, Satellite/ObservingSat21/Tr...",0.108473,6.0,3.0,3,3,2.352567e+07,0.078473,0.030
5,50,"[Target/Target7, Satellite/ObservingSat21/Sensor/ObservingFOR, Satellite/ObservingSat21, Satellite/ObservingSat21/Tr...",0.108450,6.0,3.0,3,3,2.351882e+07,0.078450,0.030
6,60,"[Target/Target7, Satellite/ObservingSat64/Sensor/ObservingFOR, Satellite/ObservingSat64, Satellite/ObservingSat64/Tr...",0.108394,6.0,3.0,3,3,2.350178e+07,0.078394,0.030
7,70,"[Target/Target7, Satellite/ObservingSat36/Sensor/ObservingFOR, Satellite/ObservingSat36, Satellite/ObservingSat36/Tr...",0.113694,6.0,3.0,2,2,2.509090e+07,0.083694,0.030
8,80,"[Target/Target7, Satellite/ObservingSat36/Sensor/ObservingFOR, Satellite/ObservingSat36, Satellite/ObservingSat36/Tr...",0.113503,6.0,3.0,2,2,2.503344e+07,0.083503,0.030
9,90,"[Target/Target7, Satellite/ObservingSat36/Sensor/ObservingFOR, Satellite/ObservingSat36, Satellite/ObservingSat36/Tr...",0.113314,6.0,3.0,2,2,2.497698e+07,0.083314,0.030


In [11]:
dfIntervals['strand'][0]

['Target/Target7',
 'Satellite/ObservingSat21/Sensor/ObservingFOR',
 'Satellite/ObservingSat21',
 'Satellite/ObservingSat21/Transmitter/ObservingTransmitter',
 'Satellite/ObservingSat83/Receiver/ObservingReceiver',
 'Satellite/ObservingSat83',
 'Satellite/RelaySat22/Sensor/RelaySatFOR',
 'Place/Washington_DC']

In [12]:
# Active objects in the network over time
objPaths = list(set((item for sublist in df['strand'] for item in sublist)))

In [13]:
# Turn on the objects in the scenario
turnGraphicsOnOff(stkRoot,objPaths,onOrOff = 'On',parentsOnly = True)

In [14]:
# Turn off the objects in the sceario
turnGraphicsOnOff(stkRoot,objPaths,onOrOff = 'Off',parentsOnly = True)

## Investigate Instances in Time

In [15]:
# Look at an instance in time (pick a time in df)
t = 0
stkRoot.CurrentTime = t

In [16]:
# Look at strand order and the node delay
objPaths = df['strand'][t/step]
nodeDelaysByStrand = {node:nodeDelaysByNode[node] for node in objPaths}
pd.DataFrame([*nodeDelaysByStrand.items()],columns=['node','nodeDelay'])

,node,nodeDelay
0,Target/Target7,0.000
1,Satellite/ObservingSat21/Sensor/ObservingFOR,0.010
2,Satellite/ObservingSat21,0.000
3,Satellite/ObservingSat21/Transmitter/ObservingTransmitter,0.005
4,Satellite/ObservingSat83/Receiver/ObservingReceiver,0.005
5,Satellite/ObservingSat83,0.000
6,Satellite/RelaySat22/Sensor/RelaySatFOR,0.010
7,Place/Washington_DC,0.000


In [17]:
#look at possible connections for the object of interest at that time
nodeInterest = objPaths[0]
possibleNodeConnections(t,nodeInterest,timesEdgesDistancesDelays)

,node1,node2,distance,timeDelay
0,Target/Target7,Satellite/ObservingSat21/Sensor/ObservingFOR,3427.049297,0.021431
1,Target/Target7,Satellite/ObservingSat36/Sensor/ObservingFOR,3166.822072,0.020563
2,Target/Target7,Satellite/ObservingSat55/Sensor/ObservingFOR,3393.680533,0.021320
3,Target/Target7,Satellite/ObservingSat64/Sensor/ObservingFOR,3427.049297,0.021431


In [18]:
# Turn on the objects in the scenario
turnGraphicsOnOff(stkRoot,objPaths,onOrOff = 'On',parentsOnly = False)

In [19]:
# Turn off the objects in the sceario
turnGraphicsOnOff(stkRoot,objPaths,onOrOff = 'Off',parentsOnly = False)

## Investigate Node Utilization

In [20]:
# Most frequnt node in the shortest path and the sum of their durations
strands = dfIntervals[['strand','start','stop']].values
dfNodesIntervals = getNodesIntervalsFromStrands(strands)
dfNodeActive = getActiveDuration(dfNodesIntervals,start,stop)
dfNodeActive.sort_values('sum dur',ascending=False).head(10)

,sum dur,% time active
node,,
Place/Washington_DC,600.0,100.000000
Target/Target7,600.0,100.000000
Satellite/ObservingSat36,535.0,89.166667
Satellite/ObservingSat36/Sensor/ObservingFOR,535.0,89.166667
Satellite/RelaySat31/Sensor/RelaySatFOR,465.0,77.500000
Satellite/ObservingSat36/Transmitter/ObservingTransmitter,365.0,60.833333
Satellite/ObservingSat75,180.0,30.000000
Satellite/ObservingSat75/Receiver/ObservingReceiver,180.0,30.000000
Satellite/ObservingSat56,160.0,26.666667


## Compute the Top N Paths for Multiple Node Pairs

In [21]:
# Loop through each node pair and compute network metrics for top N strands, assumes only 1 starting and ending node for each network
topN = 5
for nodePair in nodePairs:
    startingNode = nodePair[0]
    endingNode = nodePair[1]
    dfTopN = computeNetworkTopN(start,stop,step,timeNodePos,timesEdgesDistancesDelays,startingNode,endingNode,metric,topN=topN,overrideData=overrideNetwork,printTime=True,filename='')


0.0030012130737304688
0.0010004043579101562
0.0009970664978027344
0.0019996166229248047
0.0009999275207519531
0.0010001659393310547
0.002002716064453125
0.0009980201721191406
0.0009996891021728516
0.002000570297241211
0.002011537551879883
0.0019893646240234375
0.0019996166229248047
0.0010094642639160156


In [22]:
# Loop through each node pair and compute network metrics for top N strands, assumes only 1 starting and ending node for each network
# The strands will all be unique, but large portions of the strand may be reused unless the nodes are edges are removed
topN = 5
removeUsedNodes = False # Remove any nodes in previous shortest paths
removeUsedEdges = False # Remove any edge in previous shortest paths
for nodePair in nodePairs:
    startingNode = nodePair[0]
    endingNode = nodePair[1]
    dfTopN = computeNetworkTopN(start,stop,step,timeNodePos,timesEdgesDistancesDelays,startingNode,endingNode,metric,topN=topN,overrideData=overrideNetwork,printTime=True,filename='',removeUsedNodes=removeUsedNodes,removeUsedEdges=removeUsedEdges)


0.0029778480529785156
0.0010199546813964844
0.002008199691772461
0.0019674301147460938
0.0010006427764892578
0.0020372867584228516
0.0019736289978027344
0.0009894371032714844
0.0010349750518798828
0.0020186901092529297
0.0019812583923339844
0.0010333061218261719
0.0010008811950683594
0.0009970664978027344


## Load And Merge Multiple Networks

In [23]:
nodePairsToLoad = [
    ('Target/Target1', 'Place/Los_Angeles_CA'),
    ('Target/Target1', 'Place/Washington_DC'),
    ('Target/Target2', 'Place/Los_Angeles_CA'),
    ('Target/Target2', 'Place/Washington_DC')] # Can specifically give pairs of interest
nodePairsToLoad = nodePairs # Use all of the nodePairs
nodePairsToLoad

[('Target/Target1', 'Place/Los_Angeles_CA'),
 ('Target/Target1', 'Place/Washington_DC'),
 ('Target/Target2', 'Place/Los_Angeles_CA'),
 ('Target/Target2', 'Place/Washington_DC'),
 ('Target/Target3', 'Place/Los_Angeles_CA'),
 ('Target/Target3', 'Place/Washington_DC'),
 ('Target/Target4', 'Place/Los_Angeles_CA'),
 ('Target/Target4', 'Place/Washington_DC'),
 ('Target/Target5', 'Place/Los_Angeles_CA'),
 ('Target/Target5', 'Place/Washington_DC'),
 ('Target/Target6', 'Place/Los_Angeles_CA'),
 ('Target/Target6', 'Place/Washington_DC'),
 ('Target/Target7', 'Place/Los_Angeles_CA'),
 ('Target/Target7', 'Place/Washington_DC')]

In [24]:
# # Load multiple networks into one
# df = loadNetworkDfTopN(nodePairsToLoad,topN)
# df = addLightAndNodeDelays(df,timesEdgesDistancesDelays)    
# df

# Load multiple networks into one
df = loadNetworkDf(nodePairsToLoad)
df = addLightAndNodeDelays(df,timesEdgesDistancesDelays)    
df

,time,strand,timeDelay,num hops,num parent hops,Highest Num Nodes Removed To Lose Access,Lowest Num Nodes Removed To Lose Access,distance,lightDelay,nodeDelay
0,0,"[Target/Target1, Satellite/ObservingSat81/Sensor/ObservingFOR, Satellite/ObservingSat81, Satellite/RelaySat41/Sensor...",0.124185,5,3,2,2,27636.233937,0.092185,0.032
1,10,"[Target/Target1, Satellite/ObservingSat81/Sensor/ObservingFOR, Satellite/ObservingSat81, Satellite/RelaySat41/Sensor...",0.124094,5,3,2,2,27609.193358,0.092094,0.032
2,20,"[Target/Target1, Satellite/ObservingSat81/Sensor/ObservingFOR, Satellite/ObservingSat81, Satellite/RelaySat41/Sensor...",0.12401,5,3,2,2,27584.045432,0.092010,0.032
3,30,"[Target/Target1, Satellite/ObservingSat81/Sensor/ObservingFOR, Satellite/ObservingSat81, Satellite/ObservingSat81/Tr...",0.102295,6,3,2,2,21673.552261,0.072295,0.03
4,40,"[Target/Target1, Satellite/ObservingSat81/Sensor/ObservingFOR, Satellite/ObservingSat81, Satellite/ObservingSat81/Tr...",0.102169,6,3,2,2,21635.783066,0.072169,0.03
5,50,"[Target/Target1, Satellite/ObservingSat81/Sensor/ObservingFOR, Satellite/ObservingSat81, Satellite/ObservingSat81/Tr...",0.102053,6,3,2,2,21601.069639,0.072053,0.03
6,60,"[Target/Target1, Satellite/ObservingSat81/Sensor/ObservingFOR, Satellite/ObservingSat81, Satellite/ObservingSat81/Tr...",0.101948,6,3,2,2,21569.417183,0.071948,0.03
7,70,"[Target/Target1, Satellite/ObservingSat81/Sensor/ObservingFOR, Satellite/ObservingSat81, Satellite/ObservingSat81/Tr...",0.101852,6,3,2,2,21540.822982,0.071852,0.03
8,80,"[Target/Target1, Satellite/ObservingSat81/Sensor/ObservingFOR, Satellite/ObservingSat81, Satellite/ObservingSat81/Tr...",0.101767,6,3,2,2,21515.276545,0.071767,0.03
9,90,"[Target/Target1, Satellite/ObservingSat81/Sensor/ObservingFOR, Satellite/ObservingSat81, Satellite/ObservingSat81/Tr...",0.101692,6,3,2,2,21492.759871,0.071692,0.03


In [25]:
# Load the paths into STK, there are some basic options for how to color the network

# Create intervals for each edge and count the number of times the edge is active
timesEdgeCountAll = createTimesEdgesCountFromDF(df)

# # Consider binning the counts to prevent a ton of unique colors, which can noteably speed up time to add data to STK
# # Look at the pd.cut documentation for specifying specific bin edges
# # Alternatively it is possible to color by a custom dataset for each edge, such as approx timeDelay of the edge, this would require custom editting
# numBins = 5
# bins = pd.cut(timesEdgeCountAll[:,2],numBins,labels=False)
# timesEdgeCountAll[:,2] = bins

t1 = time.time()
addTimesEdgesCountAsObjectLines(stkRoot,timesEdgeCountAll,step,color='%000196196',useColorRamp=True,rampColor1=[255,255,255],rampColor2=[255,0,0])
time.time()-t1

8.794757604598999